In [7]:
import matplotlib.pyplot as plt
from LASSENA_Readout import LAS
folder = 'LASSENA_FLASH_time/'
from HoughTransform import HT
data, dark, OnCheck, Myst = LAS.get_data_loc(folder)

NameError: name 'folder' is not defined

In [4]:
dark_image = LAS.avg_bunch(dark)
img_for_roi = dark_image-LAS.avg_bunch(data[NEED_TO_SPECIFY]) 
#dataset with high current for finding ROI 
ROI_pts = HT.cross_points(img_for_roi,HT.hough_grid(img_for_roi))

SyntaxError: invalid syntax (<ipython-input-4-9207e4e9ecb1>, line 2)

In [8]:
#assume ROI the same for each image
#assume dark image is the same
#give list of string filenames
#going to have to know roughly which one is the FLASH frame
def get_FLASH_time(data,ROI_pts,dark_img,integration_time): 
    images = get_images(data) #get images from data locations
    warped_images = [HT.four_point_transform(image,ROI_pts) for image in images] #warp each image to consider only defined ROI
    ROI_averages = [LAS.avg_img(warped_image) for warped_image in warped_images] #calculate avg PV in ROI for each image
    [data.pop(ROI_averages.index(avg)) for avg in ROI_averages if avg<=LAS.avg_img(dark_img)]#remove any dark frames
    FLASH_frame_num = np.argmin(ROI_averages) #find smallest frame PV average in ROI = where we have FLASH frame
    FLASH_frame_AVG_PV_ROI = LAS.get_pv(data[FLASH_frame_num],ROI_pts,dark_img) #get FLASH frame avg PV in ROI
    other_data = data.pop(FLASH_frame_num) #remove FLASH frame from data
    other_frames_AVG_PV_ROI = LAS.get_pv(other_data,ROI_pts,dark_img) #get other avg PV in ROI
    FLASH_time = (FLASH_frame_AVG_PV_ROI/other_frames_AVG_PV_ROI)*integration_time #get FLASH time
    return FLASH_time

In [ ]:
FLASH_times = [get_FLASH_time(datafolder) for datafolder in data]
print(np.mean(FLASH_times))
print(np.std(FLASH_times))